In [ ]:
import os

os.environ['PYPI_USERNAME'] = ''
os.environ['PYPI_PASSWORD'] = ''

!git clone https://github.com/3lc-ai/notebook-examples.git

%run notebook-examples/tlc-install.ipynb

In [ ]:
import os

os.environ['PYPI_USERNAME'] = ''
os.environ['PYPI_PASSWORD'] = ''

!git clone https://github.com/3lc-ai/notebook-examples.git

%run notebook-examples/tlc-install.ipynb

In [ ]:
import os

os.environ['PYPI_USERNAME'] = ''
os.environ['PYPI_PASSWORD'] = ''

!git clone https://github.com/3lc-ai/notebook-examples.git

%run notebook-examples/tlc-install.ipynb

In [ ]:
import os

os.environ['PYPI_USERNAME'] = ''
os.environ['PYPI_PASSWORD'] = ''

!git clone https://github.com/3lc-ai/notebook-examples.git

%run notebook-examples/tlc-install.ipynb

In [ ]:
# Parameters
EPOCHS = 4
TRAIN_BATCH_SIZE = 64
TEST_BATCH_SIZE = 1000
LEARNING_RATE = 1.0
LEARNING_RATE_SCHEDULER_GAMMA = 0.7
EPOCHS = int(EPOCHS)
NUM_WORKERS = 4
PIN_MEMORY = True
DEVICE = "cuda"
DATASET_PATH="/datasets/mnist"

# Pytorch MNIST Example

Based on public Pytorch example from https://github.com/pytorch/examples/tree/main/mnist

In [ ]:
import torch
import torchvision
import os

import torch.nn as nn
import torch.nn.functional as F

from torchvision import transforms

In [ ]:
cuda_kwargs = {'num_workers': NUM_WORKERS,
               'pin_memory': PIN_MEMORY,
               'shuffle': True}

In [ ]:
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

In [ ]:
kwargs = {
    "train": {"batch_size": TRAIN_BATCH_SIZE},
    "val": {"batch_size": TEST_BATCH_SIZE}
}

for stage in kwargs:
    kwargs[stage].update(cuda_kwargs)

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout(0.25)
        self.dropout2 = nn.Dropout(0.5)
        self.fc1 = nn.Linear(9216, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        output = F.log_softmax(x, dim=1)
        return output

In [ ]:
device = torch.device(DEVICE)

In [ ]:
transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
        ])

In [ ]:
datasets = {
    "train": torchvision.datasets.MNIST(DATASET_PATH, train=True, download=True, transform=transform),
    "val": torchvision.datasets.MNIST(DATASET_PATH, train=False, transform=transform)
}


In [ ]:
data_loaders = {}
for stage in datasets:
    data_loaders[stage] = torch.utils.data.DataLoader(datasets[stage], **kwargs[stage])

In [ ]:
model = Net().to(device)
optimizer = torch.optim.Adadelta(model.parameters(), lr=LEARNING_RATE)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=LEARNING_RATE_SCHEDULER_GAMMA)

In [ ]:
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    total_loss = 0.0
    correct = 0
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)

        total_loss += loss.sum().item()

        loss.backward()
        optimizer.step()

        pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
        correct += pred.eq(target.view_as(pred)).sum().item()
        
    accuracy = correct / len(train_loader.dataset)
    print(f"Train - Avg. Loss: {total_loss / len(train_loader.dataset)} - Accuracy: {accuracy}")
    return {"epoch": epoch, "loss": total_loss / len(train_loader.dataset), "accuracy": accuracy, "stage": "train"}


In [ ]:
def test(model, device, loader, epoch, stage="val"):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in loader[stage]:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(loader[stage].dataset)
    accuracy = correct / len(loader[stage].dataset)

    print(f"Test - Avg. Loss: {test_loss} - Accuracy: {accuracy}")
    
    return {"epoch": epoch, "loss": test_loss, "accuracy": accuracy, "stage": stage}

In [ ]:
records = []
for epoch in range(1, EPOCHS+1):
    print(f"Epoch {epoch}:")
    train(model, device, data_loaders["train"], optimizer, epoch)
    records.append(test(model, device, data_loaders, epoch, "train"))
    records.append(test(model, device, data_loaders, epoch))
    scheduler.step()

In [ ]:
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt

fig, axs = plt.subplots(nrows=2, sharex=True)

records_df = pd.DataFrame(records)
sns.lineplot(data=records_df, x="epoch", y="accuracy", hue="stage", ax=axs[0])
sns.lineplot(data=records_df, x="epoch", y="loss", hue="stage", ax=axs[1])